## Import Libraries

In [1]:
import math
import gc
import pickle
import transformers
print(transformers.__version__)
from transformers.utils import send_example_telemetry
send_example_telemetry("language_modeling_notebook", framework="pytorch")
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
from transformers import AutoTokenizer

4.29.2


## Load Dataset

In [2]:
datasets = load_dataset(
    'csv', data_files={
        'train': '/kaggle/input/amazon-products-mlm-dataset/train.csv', 
        'test': '/kaggle/input/amazon-products-mlm-dataset/test.csv'}
)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)
/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4fa11b30ce11e40e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
datasets = datasets.remove_columns(['Unnamed: 0'])

In [4]:
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 18112435
    })
    test: Dataset({
        features: ['text'],
        num_rows: 100000
    })
})


In [5]:
# datasets = DatasetDict({
#     'train': datasets['train'].select(list(range(1000000))),
#     'test': datasets['test'].select(list(range(100000)))
# })

In [6]:
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 18112435
    })
    test: Dataset({
        features: ['text'],
        num_rows: 100000
    })
})


In [7]:
print(datasets["train"][0])

{'text': 'Only these items with mentioned quantity will be deliver Note : -The Total Quantity in the Item is Exactly the Same As Mentioned in the Title in Bracket The Content Displayed in the Photos Videos are Only Meant for the Display Usage Purpose Its recommended to provide pressurized water ( motor pump ) Since this device will not perform in normal tap water pressure'}


In [8]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(datasets["train"], 8)

,text
0,Bobby and the Bears
1,) Package Includes : 1 x Dental Model Note : 1
2,Amp Mug Bath time has suddenly become very untraditional
3,"If you ’ re like most people , you ’ ll love the comfortable feel of 100 % cotton sheets when you climb into bed each night"
4,It takes up less space
5,Each package contains 1 sticker sheet ; sheet measures 2 x 7 Inches ; 12 sheets of stickers in all
6,. Delivery within 3 to 5 weeks .
7,. Match : Can be dressed up or down


## Preprocess

In [10]:
model_checkpoint = "distilroberta-base"

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [12]:
def make_strings(examples):
    return {'text': [str(x) for x in examples['text']]}

In [13]:
print('Stringing...')

Stringing...


In [14]:
datasets = datasets.map(make_strings, batched=True, num_proc=8, batch_size=100000)

#0:   0%|          | 0/23 [00:00<?, ?ba/s]

#1:   0%|          | 0/23 [00:00<?, ?ba/s]

#2:   0%|          | 0/23 [00:00<?, ?ba/s]

#3:   0%|          | 0/23 [00:00<?, ?ba/s]

#4:   0%|          | 0/23 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

#5:   0%|          | 0/1 [00:00<?, ?ba/s]

#6:   0%|          | 0/1 [00:00<?, ?ba/s]

#7:   0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 18112435
    })
    test: Dataset({
        features: ['text'],
        num_rows: 100000
    })
})


In [16]:
def tokenize_function(examples):
    try:
        return tokenizer(examples["text"], truncation=True)
    except Exception as e:
        print(f'Exception: {e}')
        print(examples['text'])

In [17]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=8, remove_columns=["text"], batch_size=100000)

#0:   0%|          | 0/23 [00:00<?, ?ba/s]

#1:   0%|          | 0/23 [00:00<?, ?ba/s]

#2:   0%|          | 0/23 [00:00<?, ?ba/s]

#3:   0%|          | 0/23 [00:00<?, ?ba/s]

#4:   0%|          | 0/23 [00:00<?, ?ba/s]

...

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#5:   0%|          | 0/1 [00:00<?, ?ba/s]

#6:   0%|          | 0/1 [00:00<?, ?ba/s]

#7:   0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 18112435
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 100000
    })
})


In [19]:
print('Saving...')

Saving...


In [20]:
tokenized_datasets['train'].save_to_disk("train.hf")
tokenized_datasets['test'].save_to_disk("test.hf")